# DES and 3DES implementation with numpy
* Because of implementation, it is ~1000x slower then normal implementation, but is quite easy to read. Good for educational purposes.

In [ ]:
import numpy as np

# Encryption

* Uses same key for encryption and decryption
* Uses 16 rounds which all perform identical operation
* Different subkey in each round is derived from main key
* In each round $x$ is split up into 32-bit halves $L$ and $R$
<img src="img/encryption.png" alt="Drawing" style="width: 600px"/>

In [ ]:
def encrypt(x,key):
    # initial permutation
    x = IP(x)
    # 16 rounds of encryption
    k = derive_keys(key)
    L,R = np.split(x,2)
    for i in range(16):
        L,R = Round(L,R,k[i])
    x = np.concatenate((R,L))
    # final permutation
    y = FP(x)
    return y

# Initial Permutation IP

* Bitwise permutation
<img src="img/IP.png" alt="Drawing" style="width: 400px"/>

In [ ]:
IP_table = np.array([
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17,  9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
])
# indexing starts at 0, so subtract 1
IP_table -= 1

In [ ]:
def IP(x): # x=(64,)
    return permutation(x,IP_table)

# Final Permutation FP

* $FP$ is inverse of $IP$
<img src="img/FP.png" alt="Drawing" style="width: 400px"/>

In [ ]:
FP_table = np.array([
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41,  9, 49, 17, 57, 25
])
# indexing starts at 0, so subtract 1
FP_table -= 1

In [ ]:
def FP(x): # x=(64,)
    return permutation(x,FP_table)

# DES Round

* DES structure is a Feistel network
* $x$ is split up into 32-bit halves $L_i$ and $R_i$
* Each round can be expressed as: $$ L_i = R_{i-1} $$ $$ R_i = L_{i-1} \oplus f(R_{i-1},k_i) $$
<img src="img/round.png" alt="Drawing" style="width: 400px"/>

In [ ]:
def Round(L,R,k,f=f):
    return R, xor(L,f(R,k))

# The f-Function

* After round 5 every bit is a function of each key bit and each plaintext
* $k_i$ is derived from Key Schedule
<img src="img/f_function.png" alt="Drawing" style="width: 340px"/>

In [ ]:
def f(R,k):
    y = E(R)
    y = xor(y,k)
    # split up into 8 arrays of size 6
    y = np.split(y,8)
    for i in range(8):
        y[i] = S(y[i],i)
    # join all arrays into single array of size 32
    y = np.concatenate(y)
    # permute
    return P(y)

## Expansion E

* Increases diffusion
<img src="img/expansion.png" alt="Drawing" style="width: 400px"/>

In [ ]:
E_table = np.array([
    32,  1,  2,  3,  4,  5,
     4,  5,  6,  7,  8,  9,
     8,  9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32,  1
])
# indexing starts at 0, so subtract 1
E_table -= 1

In [ ]:
def E(x): # x=(32,)
    return permutation(x,E_table)

## Permutation P

* Introduces diffusion

In [ ]:
P_table = np.array([
    16,  7, 20, 21, 29, 12, 28, 17,
     1, 15, 23, 26,  5, 18, 31, 10,
     2,  8, 24, 14, 32, 27,  3,  9,
    19, 13, 30,  6, 22, 11,  4, 25
])
P_table -= 1

In [ ]:
def P(x):
    return permutation(x,P_table)

## DES S-Boxes

* 6 bits input, 4 bits output
* Resistant to differential cryptanalysis
<img src="img/s_box_.png" alt="Drawing" style="width: 450px"/>

In [ ]:
S_table = np.zeros((8,4,16),dtype=int)
# S_table=(number of tables,rows,columns)
S_table[0] = np.array([
           [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
           [ 0,  15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3, 8],
           [ 4,   1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5, 0],
           [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
])
S_table[1] = np.array([
           [15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
           [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
           [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
           [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]
])
S_table[2] = np.array([
           [10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
           [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
           [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
           [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]
])
S_table[3] = np.array([
           [ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
           [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
           [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
           [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]
])
S_table[4] = np.array([
           [ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
           [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
           [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
           [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]
])
S_table[5] = np.array([
           [12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
           [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
           [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
           [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]
])
S_table[6] = np.array([
           [ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
           [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
           [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
           [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]
])
S_table[7] = np.array([
           [13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
           [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
           [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
           [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]
])

In [ ]:
def S(x,i): # x=(6,)
    # create bit array of first element and last
    row_index_bin = [ bit for i,bit in enumerate(x) if i in [0,5] ]
    # create bit array of middle elements
    column_index_bin = [ bit for i,bit in enumerate(x) if i in [1,2,3,4] ]
    # get decimal value from table
    dec_val = S_table[ i, bit_array2decimal(row_index_bin), bit_array2decimal(column_index_bin) ]
    # convert to bit array
    return decimal2bit_array(dec_val)

# Key schedule

* Derives 16 pairs of 48 bit keys
* $PC-1$ removes parity bits $(8,16,24,32,40,48,56,64)$
* In rounds $i=1,2,9,16$: $C$ and $D$ are rotated left by one bit
* In all other rounds: $C$ and $D$ are rotated left by two bits
* Each $k_i$ is a permutation of $k$
* $C_0$ = $C_{16}$ and $D_0$ = $D_{16}$
<img src="img/key_schedule.png" alt="Drawing" style="width: 600px"/>

In [ ]:
PC1_table = np.array([
    57, 49, 41, 33, 25, 17,  9,
     1, 58, 50, 42, 34, 26, 18,
    10,  2, 59, 51, 43, 35, 27,
    19, 11,  3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
     7, 62, 54, 46, 38, 30, 22,
    14,  6, 61, 53, 45, 37, 29,
    21, 13,  5, 28, 20, 12,  4
])
PC2_table = np.array([
    14, 17, 11, 24,  1,  5,  3, 28,
    15,  6, 21, 10, 23, 19, 12,  4,
    26,  8, 16,  7, 27, 20, 13,  2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
])
# indexing starts at 0, so subtract 1
PC1_table -= 1
PC2_table -= 1

In [ ]:
def init_transform(k):
    ''' k=(64,) output: 2*(28,) '''
    k = permutation(k,PC1_table)
    return k[:28],k[28:]

In [ ]:
def transform(C,D,i,shift=left_shift):
    ''' C,D=(28,), i=round number, n=how many bits to rotate '''
    # n is 1 if round is 1,2,9,16 else n is 2 
    n = 1 if i in [0,1,8,15] else 2
    # shift C,D by n
    C, D = shift(C,n), shift(D,n)
    # permute C,D to get key
    C_and_D = np.concatenate((C,D))
    k = permutation(C_and_D,PC2_table)
    return C,D,k

In [ ]:
def derive_keys(Key):
    ''' Derives 16 keys of 48 bit length out of Key(64 len) '''
    k = np.zeros((16,48),dtype=int)
    C,D = init_transform(Key)
    for i in range(16):
        # i = [0,1,...,15]
        C,D,k[i] = transform(C,D,i)
    return k

# Decryption

* Is the same as encryption, but keys are in reverse order

In [ ]:
def decrypt(y,key):
    # initial permutation
    y = IP(y)
    # 16 rounds of encryption
    k = derive_keys(key)
    L,R = np.split(y,2)
    for i in range(16)[::-1]:
    # i = [15,14,...,0]
         L,R = Round(L,R,k[i])
    y = np.concatenate((R,L))
    # final permutation
    x = FP(y)
    return x

# Building blocks

In [ ]:
def xor(A,B):
    ''' input: bit array A and B, output: A xor B  '''
    return 1*np.logical_xor(A,B)

In [ ]:
def left_shift(arr,n):
    ''' performs circular left shift '''
    return np.roll(arr,-n)

In [ ]:
def right_shift(arr,n):
    ''' performs circular right shift '''
    return np.roll(arr,n)

In [ ]:
def permutation(x,table):
    ''' permute x -> y, where len(y)=len(table) '''
    y = np.zeros(len(table),dtype=int)
    for output_index, input_index in enumerate(table):
        y[output_index] = x[input_index]
    return y

In [ ]:
def bit_array2decimal(bit_arr):
    ''' converts bit array to decimal number '''
    # reverse bit array
    bit_arr = np.flip(bit_arr,axis=0)
    # create array [1,2,4,8,...]
    power_arr = 2**(np.arange( len(bit_arr) ))
    # apply mask of bit_array to power_arr
    y = power_arr * bit_arr
    return y.sum()

In [ ]:
def decimal2bit_array(decimal,width=4):
    ''' converts decimal(0-15) to 4 (width) bits '''
    # convert to binary string
    bin_str = np.binary_repr(decimal, width=width)
    # convert string to bit array
    return np.array([ int(bit) for bit in bin_str ],dtype=int)

# File reading

In [ ]:
def read_file(file_name):
    file_in_bytes = np.fromfile(file_name, dtype = "uint8")
    file_in_bits = np.unpackbits(file_in_bytes)
    return file_in_bits

In [ ]:
def write_file(file_name,file_in_bits):
    file_in_bytes = np.packbits(file_in_bits)
    file_in_bytes.tofile(file_name)
#     equivalent to
#     with open(file_name,'w') as f:
#         f.write(file_in_bytes.tobytes())

# PKCS5 Padding

* The last bytes are padded so the file could be split into even blocks of 64 bits
* PKCS5 padding: If the block length is $B$ then add $N$ padding bytes of value $N$ to make the input length up to the next exact multiple of $B$.
* Examples: <br>
  3 bytes: $FDFDFD.................. \longrightarrow FDFDFD0505050505$ <br>
  7 bytes: $FDFDFDFDFDFDFD....   \longrightarrow FDFDFDFDFDFDFD01$ <br>
  8 bytes: $FDFDFDFDFDFDFDFD \longrightarrow FDFDFDFDFDFDFDFD0808080808080808$ <br>

In [ ]:
def pad_file(file_in_bits):
    # count left unused bits (64 - size of block (B))
    n = file_in_bits.size%64
    # count left bytes (8 bytes in 1 block). 8 bits in 1 byte.
    N = int( 8 - np.ceil(n/8) )
    # add N times N number (in byte form) to file
    for i in range(N):
        byte_in_bits = decimal2bit_array(N,width=8)
        file_in_bits = np.append( file_in_bits, byte_in_bits, axis=0 )
    return file_in_bits

In [ ]:
def unpad_file(file_in_bits):
    # read last byte
    last_byte_in_bits = file_in_bits[-8:]
    # convert to decimal
    l = bit_array2decimal(last_byte_in_bits)
    # remove padded bits
    n = file_in_bits.size
    return np.resize(file_in_bits,n-(l*8))

# File Encryption and Decryption in ECB mode

* Each block is encrypted separately

In [ ]:
def encrypt_file(file_in_bits,key):
    # split into blocks of 64
    no_blocks = int(file_in_bits.size/64)
    blocks = file_in_bits.reshape((no_blocks,64))
    # encrypt all blocks
    encrypted_file = []
    for i in range(no_blocks):
        x = blocks[i]
        y = encrypt(x,key)
        encrypted_file.extend(y)
        print('Encrypting blocks: {}/{} '.format(i, no_blocks), end='\r')
    print('')
    return np.array(encrypted_file)

In [ ]:
def decrypt_file(file_in_bits,key):
    # split into blocks of 64
    no_blocks = int(file_in_bits.size/64)
    blocks = file_in_bits.reshape((no_blocks,64))
    # decrypt file
    decrypted_file = []
    for i in range(no_blocks):
        y = blocks[i]
        x = decrypt(y,key)
        decrypted_file.extend(x)
        print('Decrypting blocks: {}/{} '.format(i, no_blocks), end='\r')
    print('')
    return np.array(decrypted_file)

# DES and 3DES

* 3DES is DES encryption 3 times: $$ y = e_{k3}( e_{k2}( e_{k1}(x) ) ) $$ $$ x = d_{k1}( d_{k2}( d_{k3}(y) ) ) $$

In [ ]:
def des_enc(input_name,output_name,key):
    # read file
    file_in_bits = read_file(input_name)
    # pad file
    file_in_bits = pad_file(file_in_bits)
    # encrypt
    encrypted_file = encrypt_file(file_in_bits,key)
    # save file
    write_file(output_name,encrypted_file)
    
def des_dec(input_name,output_name,key):
    # read file
    file_in_bits = read_file(input_name)
    # decrypt
    decrypted_file = decrypt_file(file_in_bits,key)
    # unpad file
    decrypted_file = unpad_file(decrypted_file)
    # save file
    write_file(output_name,decrypted_file)

In [ ]:
def tri_des_enc(input_name,output_name,key):
    # split key into 3 keys
    key = key.reshape((3,64))
    # read file
    file_in_bits = read_file(input_name)
    # pad file
    file_in_bits = pad_file(file_in_bits)
    # encrypt
    encrypted_file = file_in_bits
    for i in range(3):
        encrypted_file = encrypt_file(encrypted_file,key[i])
    # save file
    write_file(output_name,encrypted_file)

def tri_des_dec(input_name,output_name,key):
    # split key into 3 keys
    key = key.reshape((3,64))
    # read file
    file_in_bits = read_file(input_name)
    # decrypt
    decrypted_file = file_in_bits
    for i in range(3)[::-1]:
        decrypted_file = decrypt_file(decrypted_file,key[i])
    # unpad file
    decrypted_file = unpad_file(decrypted_file)
    # save file
    write_file(output_name,decrypted_file)

# Testing

In [ ]:
key = np.random.randint(2, size=64)
des_enc('test.png','test.des',key)
des_dec('test.des','test2.png',key)

f1 = read_file('test.png')
f2 = read_file('test2.png')
np.array_equal(f1,f2)

In [ ]:
key = np.random.randint(2, size=192)
tri_des_enc('test.png','test.3des',key)
tri_des_dec('test.3des','test3.png',key)

f1 = read_file('test.png')
f2 = read_file('test3.png')
np.array_equal(f1,f2)